**Imports**

In [1]:
# Imports
import pandas as pd
import random
import numpy as np

**Dataset imports**

In [2]:
train_df = pd.read_csv(r'training.csv')
test_df = pd.read_csv(r'testing.csv')
# Printing the column headings to better understand the features
print(train_df.columns.values)

['Duration' 'Distance' 'Pickup_longitude' 'Pickup_latitude'
 'Dropoff_latitude' 'Dropoff_latitude.1' 'Haversine' 'Pmonth' 'Pickup_day'
 'Pickup_hour' 'Pickup_minute' 'Pickup_weekday' 'Dropoff_month'
 'Dropoff_day' 'Dropoff_hour' 'Dropoff_minute' 'Dropoff_weekday' 'Temp'
 'Precip' 'Wind' 'Humid' 'Solar' 'Snow' 'GroundTemp' 'Dust']


In [3]:
len(train_df.columns.values)

25

In [44]:
from deap import gp, creator, base, tools, algorithms
import operator

In [56]:
def protectedDiv(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 23)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
# pset.addPrimitive(protectedDiv, 2)
# pset.addPrimitive(operator.neg, 1)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin, pset=pset)

c:\Users\david\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\david\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [57]:
# Toolbox setup
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [58]:
def fitness_function(individual):
    # print("evaluating fitness...")
    # print("individual:")
    # print(type(individual))
    trip_sample = train_df.sample(100, ignore_index=True, random_state=10)

    # print(trip_sample.columns.values)

    # break into iteratable arrays
    durations = trip_sample.loc[:,"Duration"]
    distances = trip_sample.loc[:,"Distance"]
    pickup_longs = trip_sample.loc[:,"Pickup_longitude"]
    pickup_lats = trip_sample.loc[:,"Pickup_latitude"]
    dropoff_longs = trip_sample.loc[:,"Dropoff_latitude.1"]
    dropoff_lats = trip_sample.loc[:,"Dropoff_latitude"]
    haversines = trip_sample.loc[:,"Haversine"]
    pmonths = trip_sample.loc[:,"Pmonth"]
    pdays = trip_sample.loc[:,"Pickup_day"]
    phours = trip_sample.loc[:,"Pickup_hour"]
    pmins = trip_sample.loc[:,"Pickup_minute"]
    pweekdays = trip_sample.loc[:,"Pickup_weekday"]
    ddays = trip_sample.loc[:,"Dropoff_day"]
    dhours = trip_sample.loc[:,"Dropoff_hour"]
    dmins = trip_sample.loc[:,"Dropoff_minute"]
    dweekdays = trip_sample.loc[:,"Dropoff_weekday"]
    temps = trip_sample.loc[:,"Temp"]
    precips = trip_sample.loc[:,"Precip"]
    winds = trip_sample.loc[:,"Wind"]
    humids = trip_sample.loc[:,"Humid"]
    solars = trip_sample.loc[:,"Solar"]
    snows = trip_sample.loc[:,"Snow"]
    groundtemps = trip_sample.loc[:,"GroundTemp"]
    dusts = trip_sample.loc[:,"Dust"]

    func = toolbox.compile(expr=individual)
    # print("function compiled")
    fitness = 0
    for i in range(len(trip_sample)):
        # print("fitness eval loop run " + str(i))
        # PSUEDOCODE: individual_fitness = abs(correct_duration - calculated_duration)
        # PSEUDOCODE: fitness = fitness + individual_fitness
        sample_row = trip_sample.iloc[[i]]
        # print("sample row: " + str(type(sample_row)))
        sample_features = sample_row.drop("Duration",axis=1)
        # print("sample features: " + str(type(sample_features)))

        correct_duration = durations[i]
        # print("correct duration for item " + str(i) + " is " + str(correct_duration))
        calculated_duration = func(
            distances[i], pickup_longs[i], pickup_lats[i], dropoff_longs[i], dropoff_lats[i],
            haversines[i], pmonths[i], pdays[i], phours[i], pmins[i], pweekdays[i], ddays[i],
            dhours[i], dmins[i], dweekdays[i], temps[i], precips[i], winds[i], humids[i], solars[i],
            snows[i], groundtemps[i], dusts[i]
        )
        # print("calculated duration for item " + str(i) + " is " + str(calculated_duration))

        individual_fitness = abs(correct_duration - calculated_duration)
        fitness = fitness + individual_fitness
    # print("Fitness: " + str(fitness))
    return fitness,

In [59]:
toolbox.register("evaluate", fitness_function)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [60]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [62]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.5, stats=mstats,
                                   halloffame=hof, verbose=True, ngen=20)

   	      	                       fitness                        	                      size                     
   	      	------------------------------------------------------	-----------------------------------------------
gen	nevals	avg   	gen	max        	min    	nevals	std   	avg 	gen	max	min	nevals	std    
0  	100   	282036	0  	5.64092e+06	1624.56	100   	974230	4.16	0  	7  	3  	100   	1.74768
1  	67    	7.60328e+06	1  	6.77078e+08	1624.56	67    	6.74988e+07	4.88	1  	13 	1  	67    	2.78309
2  	76    	497238     	2  	1.69907e+07	1672   	76    	2.39089e+06	5.14	2  	15 	1  	76    	3.03651
3  	83    	371127     	3  	2.41596e+07	1651   	83    	2.49225e+06	4.44	3  	13 	1  	83    	3.1123 
4  	80    	148505     	4  	1.37092e+07	1649   	80    	1.36352e+06	4.6 	4  	13 	1  	80    	3.29848
5  	80    	168615     	5  	9.15631e+06	1649   	80    	1.05557e+06	4.58	5  	13 	1  	80    	3.08927
6  	74    	2.35089e+07	6  	2.34778e+09	1649   	74    	2.33598e+08	4.44	6  	17 	1  	74    	3.00772
7  	78  